#Imports

In [0]:
%pip install yfinance

In [0]:
import yfinance as yf
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import year,month

#ingesta de datos

In [0]:

SP500_UNIVERSE = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "BRK-B",
    "JPM", "JNJ", "V", "PG", "UNH", "HD", "MA", "DIS", "PYPL", "BAC",
    "NFLX", "ADBE"
]

def ingest_ticker(ticker: str, start: str, end: str) -> pd.DataFrame:
    data = yf.download(ticker, start=start, end=end, auto_adjust=True)
    data = data.reset_index()
    data.columns = [c[0].lower().replace(" ", "_") for c in data.columns]
    data["ticker"] = ticker
    return data

def transform(prices_pdf: pd.DataFrame):
    prices_sdf = spark.createDataFrame(prices_pdf)
    return (prices_sdf.withColumn("year",year(col("date")))
                .withColumn("month",month(col("date")))
                .withColumn("date",to_date(col("date"))))
def load(prices_sdf: DataFrame, path: str):
    (prices_sdf.write
                .option("overwriteSchema", "True")
                .format("delta")
                .mode("overwrite")
                .partitionBy("ticker","year")
                .save(path))
    
prices_df = pd.DataFrame()
print(f"Tipo de prices_df antes del bucle: {type(prices_df)}")
for ticker in SP500_UNIVERSE:
    df = ingest_ticker(ticker, "2021-01-01", "2024-12-31")
    prices_df = pd.concat([prices_df,df])
transformed = transform(prices_df)
load(transformed, "/Volumes/market-mood/processed/prices_clean")



In [0]:
# SP500_UNIVERSE = [
#     "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "BRK-B",
#     "JPM", "JNJ", "V", "PG", "UNH", "HD", "MA", "DIS", "PYPL", "BAC",
#     "NFLX", "ADBE"
# ]


# def ingest_ticker(ticker: str, start: str, end: str) -> pd.DataFrame:
#     data = yf.download(ticker, start=start, end=end, auto_adjust=True)
#     data = data.reset_index()
#     data.columns = [c[0].lower().replace(" ", "_") for c in data.columns]
#     data["ticker"] = ticker
#     return data

    

# def extract(tickers: list, start: str, end: str) -> pd.DataFrame:
#     prices_df = pd.DataFrame()
#     print(f"Tipo de prices_df antes del bucle: {type(prices_df)}")
#     for ticker in tickers:
#         df = ingest_ticker(ticker, start, end)
#         prices_df = pd.concat([prices_df,df])
#     return prices_df

# def transform(prices_pdf: pd.DataFrame):
#     prices_sdf = spark.createDataFrame(prices_pdf)
#     return (prices_sdf.withColumn("year",year(col("date")))
#                 .withColumn("month",month(col("date")))
#                 .withColumn("date",to_date(col("date"))))



# def load(prices_sdf: DataFrame, path: str):
#     (prices_sdf.write
#                 .option("overwriteSchema", "True")
#                 .format("delta")
#                 .mode("overwrite")
#                 .partitionBy("ticker","year")
#                 .save(path))

# # Ejecución principal
# if __name__ == "__main__":
#     raw = extract(SP500_UNIVERSE, "2021-01-01", "2024-12-31")
#     transformed = transform(raw)
#     load(transformed, "/Volumes/market-mood/raw/prices")